In [158]:
import torch
import torch.nn as nn

experts = nn.ModuleList([nn.Linear(128, 10) for _ in range(5)])
experts

ModuleList(
  (0-4): 5 x Linear(in_features=128, out_features=10, bias=True)
)

In [159]:
router_output = torch.Tensor([[0.5696, 0.4304, 0.0000],
                               [0.0000, 0.2981, 0.7019]])

router_output

tensor([[0.5696, 0.4304, 0.0000],
        [0.0000, 0.2981, 0.7019]])

In [160]:
topk_indices = torch.Tensor([[0, 1],
                             [2, 1]])
topk_indices

tensor([[0., 1.],
        [2., 1.]])

In [161]:
x = torch.rand((2, 128))

In [162]:
embedding_mask = (topk_indices == 2)
embedding_mask

tensor([[False, False],
        [ True, False]])

In [163]:
embedding_mask.any(dim=1)

tensor([False,  True])

In [164]:
selected_embedding = x[embedding_mask.any(dim=1)]
selected_embedding.shape

torch.Size([1, 128])

In [165]:
expert_output = experts[0](selected_embedding)
expert_output.shape

torch.Size([1, 10])

In [166]:
weighted_gate = router_output[:, 2][embedding_mask.any(dim=1)]
weighted_gate

tensor([0.7019])

In [167]:
final_output = torch.zeros((2, 10))
final_output.shape

torch.Size([2, 10])

In [168]:
final_output[embedding_mask.any(dim=1)]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [169]:
final_output[embedding_mask.any(dim=1)] =  expert_output * weighted_gate.unsqueeze(1)

In [170]:
final_output

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0911,  0.4237,  0.1218,  0.2765,  0.1078,  0.1962,  0.0067,  0.0514,
         -0.3243, -0.2725]], grad_fn=<IndexPutBackward0>)